<div align=center>
		
<p></p>
<p></p>
<font size=5>
In the Name of God
<font/>
 <p></p>
 <br/>
		<div align=center>
		    <size=6>
			    <br />
Practical Assignment 4 - part 1
              <font color=blue size=6>
            	<br/>
              Naive Bayes Classifier
			</font>
    <br/>
    <br/>
 </div>
<hr/>
Artifical Intelligence - Dr. GholamReza GhassemSani
</font>
  <p></p>
 <br/>
Sharif University of Technology
<p></p>
Spring 2022


<br/>
<br/>
<font size=4 color=red>
 </font>
                <br/>
    </div>

**Due date: 19/4/1401 (at 11:59pm)**<br>
You are free to collaborate but solutions must be written up individually.
Collaborators **must** be acknowledged.<br>
Submissions with more than 100 hours delay will not be graded.<br>Submissions with less than
50 hours delay will be penalized by the following rule:<br>
**Penalized mark = M * (100 – D) / 100** <br>
Where M = the mark achieved from your solution and D is number of hours passed the
deadline. Submissions with 50 < X ≤ 100 hours delay will be penalized by P.M. = M * 0.5.<br>
Submit your answers on quera.

student_number = 400211546
Name = Danial
Last_Name = Ahangarani

# Introduction

### In this assignment, you are going to learn about the Naive Bayes algorithm, including how it works and how to implement it from scratch on a given dataset. Hence you are not allowed to use scikit-learn for implementing NB.
### You will use the [SentiPers](https://github.com/phosseini/SentiPers) dataset for this assignment, which is a sentiment analysis corpus for Persian.
### Note that F1-Score will be used to evaluate your code.


# SentiPers Dateset

### You can download the dataset from this link:
https://github.com/phosseini/SentiPers/blob/master/data/sentipers.xlsx
### The SentiPers dataset involves polarity of some sentences. There are more than 15,500 unique sentences with polarity labels in the range of ["-2", "-1", "0", "+1", "+2"] with -2 being very negative, +2 being very positive, and 0 being neutral.
### In this step, you have to import the necessary dependencies, download the dataset and do some initial preprocessing on the dataset.


### Do not forget the following items: 

1.   Download the dataset and show some of its rows.
2.   Briefly describe the dataset. Check that if it has any Nan or duplicated values.
3.   Describe the polarity column. Show its unique values and count each one.
4.   Ignore all sentences with 0 as their polarity label. Also, assume that there are only two positive and negative classes, +1 and +2 as positive and -1 and -2 as negative.
5.   (Optional) Text preprocessing is the very first step of NLP projects. You can either implement this part, or use libraries ([hazm](https://github.com/sobhe/hazm), [parsivar](https://github.com/ICTRC/Parsivar)). Use appropriate preprocessing steps based on the dataset.
6.   Report the corpus's total number of sentences and words. Also, print the ten most seen words with their frequency.
7.   Note that the dataset is imbalanced (the positive and negative classes are not represented equally). Explain how this problem can affect the accuracy and what your solution is.

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import Counter

In [26]:
#!git clone https://github.com/phosseini/SentiPers.git

In [4]:
df = pd.read_excel('sentipers.xlsx')
df[:5]

,index,sid,text,polarity,file
0,0,rev-1,اینک قصد داریم پرینتر دیگری از پرینترهای لیزری...,0,data/main/HP LaserJet M1132.xml
1,1,rev-2,پرینتری چند کاره از رده‌ی Entry Level یا سطح م...,0,data/main/HP LaserJet M1132.xml
2,2,rev-3,به هر صورت اکنون ما در دنیایی زندگی می‌کنیم، ...,0,data/main/HP LaserJet M1132.xml
3,3,rev-4,به صورتی که توانایی کپی کردن، اسکن، فکس، پر...,0,data/main/HP LaserJet M1132.xml
4,4,rev-5,به هر صورت معمولا چیزی که بیشتر کاربران از پری...,2,data/main/HP LaserJet M1132.xml


In [28]:
#دیتاست شامل نظرات کاربران دیجی کالا بر محصولات و برچسبی است
#که بر هر نظر بر یک محصول، در غالب ستون پولاریتی زده شده است
# .که از بازه دو مثبت تا دو منفی است
#
#ارزش دو مثبت در ستون پولاریتی به مثبت ترین نظرات، ارزش یک مثبت 
#به نظرات کمتر مثبت، ارزش صفر به نظرات خنثی و همین طور ارزش  
#یک منفی و دو منفی به بسته به میزان منفی بودن یک نظر کاربر بر یک
#.محصول داده شده است

In [5]:
# TODO
#Checking if dataset has duplicated values or null values.
null_check = df.isnull().values.any()
dup_check = df.duplicated().any()
print(f"data set has null values: {null_check}")
print(f"data set has duplicated values: {dup_check}")
print("*************************************")

#Showing polarity unique values and counting each one.
for i in sorted(pd.unique(df['polarity'])):
    print(f"data set has {len(df[df['polarity'] == i])} with {i} polarity.")

#Ignoring polarity = 0 and merging classes.
df = df[df.polarity != 0]
df['P/N'] = np.where((df['polarity']==1) | (df['polarity']==2) , 'P', 'N')


data set has null values: False
data set has duplicated values: False
*************************************
data set has 212 with -2 polarity.
data set has 1574 with -1 polarity.
data set has 5938 with 0 polarity.
data set has 5019 with 1 polarity.
data set has 2940 with 2 polarity.


In [6]:
#Normalizing and tokenizing using parsivar library.
from parsivar import *
normalizer = Normalizer()
df_normalized_text = []
df_tokenized_text = []
for t in df['text']:
    df_normalized_text.append(normalizer.normalize(t))
tokenizer = Tokenizer()
for t in df_normalized_text:
    df_tokenized_text.append(tokenizer.tokenize_words(t))


In [7]:
#carpus sentences and words. 
sentences = []
sum_sen = 0
for sen in df_normalized_text:
    sentences.append(tokenizer.tokenize_sentences(sen))
for s in sentences:
    sum_sen += len(s)
print(f"number of sentences: {sum_sen}")
word = 0
all_word = []
for w in df_tokenized_text:
    word += len(w)
    all_word += w
print(f"number of words: {word}")

c = Counter(all_word)
ten_most = c.most_common(10) 
print("\n")
print("ten most seen words in corpus:")
for t in range(len(ten_most)):
    print(f"{t+1}: {ten_most[t][0]}")


number of sentences: 10494
number of words: 226633


ten most seen words in corpus:
1: .
2: و
3: به
4: که
5: این
6: از
7: در
8: ،
9: با
10: است


In [200]:
# 1786 neg and 7959 pos.
#دیتاست بالانس نیست و روی داده تست
#نظر + تقریبا چهار برابر نظر - است
#حتی اگر مدل نتواند هیچ نظر منفی را پیش بینی کند باز هم
#Accuracy 
#هشتاد در صد است چون دیتا شامل همین درصد نظر مثبت است
#پس
#Accuracy 
#معیار خوبی برای دیتای غیر بالانس نیست
# we can use F1 Score, which is a function of Recall and Precision.
#اما من در اینجا با 
#Resampling
#تعداد سمپل های گروه مثبت و منفی را برابر در نظر گرفتم

In [8]:
from parsivar import *
ddf = pd.read_excel('sentipers.xlsx')
ddf = ddf[ddf.polarity != 0]
ddf['P/N'] = np.where((ddf['polarity']==1) | (ddf['polarity']==2) , 'P', 'N')
del ddf['index']
del ddf['sid']
del ddf['polarity']
del ddf['file']

ddf = ddf.sample(frac=1,random_state=4)
ndf = ddf.loc[ddf['P/N'] == 'N']
pdf = ddf.loc[ddf['P/N'] == 'P'].sample(n=len(ndf),random_state=42)
ddf = pd.concat([pdf, ndf])
ddf = ddf.sample(frac=1,random_state=4)

data_randomized = ddf.sample(frac=1, random_state=1)
training_test_index = round(len(data_randomized)*.75)

training_set = data_randomized[:training_test_index].reset_index(drop=True)
test_set = data_randomized[training_test_index:].reset_index(drop=True)

normalizer = Normalizer()
temp1 = []
for t in training_set['text']:
    temp1.append(normalizer.normalize(t))
training_set['text'] = temp1
tokenizer = Tokenizer()
temp2 = []
for t in training_set['text']:
    temp2.append(tokenizer.tokenize_words(t))
training_set['text'] = temp2

vocabulary = []
for text in training_set['text']:
    for word in text:
        vocabulary.append(word)
vocabulary = list(set(vocabulary))

# Feature Extraction

### In this step, you must extract good features from the raw text and prepare it for the Naive Bayes algorithm. A simple way is to transform a given text into a vector on the basis of the frequency (count) of each word that occurs in the entire text.
### Note that extracting good and meaningful features greatly affects the model and can get you extra credit.

In [9]:
# TODO
word_counts_per_text = {unique_word: [0] * len(training_set['text']) for unique_word in vocabulary}
for index, t in enumerate(training_set['text']):
    for word in t:
        word_counts_per_text[word][index] += 1
word_counts = pd.DataFrame(word_counts_per_text)
training_set_clean = pd.concat([training_set, word_counts], axis=1)

# Naive Bayes

### Implement Naive Bayes from scratch!

In [10]:
# Isolating p and n
positives = training_set_clean[training_set_clean['P/N'] == 'P']
negetives = training_set_clean[training_set_clean['P/N'] == 'N']

# P(positive) and P(negetive)
p_p = len(positives) / len(training_set_clean)
p_n = len(negetives) / len(training_set_clean)

# N_P
n_words_per_pos = positives['P/N'].apply(len)
n_p = n_words_per_pos.sum()

# N_N
n_words_per_neg = negetives['P/N'].apply(len)
n_n = n_words_per_neg.sum()

# N_Vocabulary
n_vocabulary = len(vocabulary)

# Laplace smoothing
alpha = 1

# Initiate parameters
parameters_pos = {unique_word:0 for unique_word in vocabulary}
parameters_neg = {unique_word:0 for unique_word in vocabulary}

# Calculate parameters
for word in vocabulary:
    n_word_given_pos = positives[word].sum() 
    p_word_given_pos = (n_word_given_pos + alpha) / (n_p + alpha*n_vocabulary)
    parameters_pos[word] = p_word_given_pos

    n_word_given_neg = negetives[word].sum() 
    p_word_given_neg = (n_word_given_neg + alpha) / (n_n + alpha*n_vocabulary)
    parameters_neg[word] = p_word_given_neg

In [11]:
def classify(message):
    message = tokenizer.tokenize_words(normalizer.normalize(message))
    p_pos_given_message = p_p
    p_neg_given_message = p_n

    for word in message:
        if word in parameters_pos:
            p_pos_given_message *= parameters_pos[word]

        if word in parameters_neg: 
             p_neg_given_message *= parameters_neg[word]

    if p_neg_given_message > p_pos_given_message:
        return 'N'

    else:
        return 'P'

In [12]:
l = []
for t in test_set['text']:
    l.append(classify(t))

# Evaluation

### Your code will be evaluated by the F1 score. If your model predicts better than a random predictor, you will get the total score of this part. So explain what F1 score we get with a random predictor.

### After beating the random predictor, you will get extra credit for improving your model due to your F1 score.

### Use 75% of the data for the training and 25% for the testing set.

### Do not forget to report the following items:

1.   Accuracy
2.   Precision
3.   Recall
4.   Confusion Matrix
5.   F1 Score



In [24]:
from sklearn.model_selection import train_test_split

In [ ]:
# TODO: Do not forget to extract features. You can also do this step after the train-test split.

In [17]:
true_num = 0
tp = 0
fp = 0
fn = 0
tn = 0
for a,b in zip(l,test_set['P/N']):
    if a == b:
         true_num += 1
    if a == 'P' and b == 'P':
        tp += 1
    if a == 'P' and b == 'N':
        fp += 1
    if a == 'N' and b == 'P':
        fn += 1
    if a == 'N' and b == 'N':
        tn += 1
        
accuracy = true_num/len(l)  
precision = tp/(tp+fp)
recall = tp/(tp+fn)
F1 = (2*recall*precision)/(precision+recall)

print(f'accuracy: {accuracy}')
print(f'precision: {precision}')
print(f'recall: {recall}')
print(f'F1: {F1}')
c_m = [ (tp, fp) ,(fn, tn) ]
confusion_matrix = pd.DataFrame(c_m, columns = ['Assigned Positive', 'Assigned Negetive'], index=['True Positive', 'True Negetive'])
confusion_matrix

accuracy: 0.799552071668533
precision: 0.8145539906103286
recall: 0.7762863534675615
F1: 0.7949599083619702


,Assigned Positive,Assigned Negetive
True Positive,347,79
True Negetive,100,367


# scikit-learn

### Now use Scikit-learn and compare it with your model. You can use CountVectorizer for extracting features and MultinomialNB for the NB model. Feel free to use other methods.

In [32]:
# Try not to change this cell.
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
labels = df['P/N']
X = df['text']
x, x_test, y, y_test = train_test_split(X, labels, stratify=labels, test_size=0.25, random_state=42)
 
# Create a Vectorizer Object
vectorizer = CountVectorizer()
 
vectorizer.fit(x)
 
# Encode the Document
vector = vectorizer.transform(x)
vector1 = vectorizer.transform(x_test)
  
clf = MultinomialNB()
clf.fit(vector, y)

S_pre = clf.predict(vector1)

In [34]:
true_num_S = 0
tp_S = 0
fp_S = 0
fn_S = 0
tn_S = 0
for a,b in zip(S_pre,y_test):
    if a == b:
         true_num_S += 1
    if a == 'P' and b == 'P':
        tp_S += 1
    if a == 'P' and b == 'N':
        fp_S += 1
    if a == 'N' and b == 'P':
        fn_S += 1
    if a == 'N' and b == 'N':
        tn_S += 1

In [35]:
accuracy_S = true_num_S/len(S_pre)  
precision_S = tp_S/(tp_S+fp_S)
recall_S = tp_S/(tp_S+fn_S)
F1_S = (2*recall_S*precision_S)/(precision_S+recall_S)

print(f'accuracy: {accuracy_S}')
print(f'precision: {precision_S}')
print(f'recall: {recall_S}')
print(f'F1: {F1_S}')
c_m_S = [ (tp_S, fp_S) ,(fn_S, tn_S) ]
confusion_matrix_S = pd.DataFrame(c_m_S, columns = ['Assigned Positive', 'Assigned Negetive'], index=['True Positive', 'True Negetive'])
confusion_matrix_S

accuracy: 0.8395568321707017
precision: 0.8853012048192771
recall: 0.9231155778894472
F1: 0.9038130381303813


,Assigned Positive,Assigned Negetive
True Positive,1837,238
True Negetive,153,209
